In [1]:
import ROOT

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.cc","k")

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.cc","k")

date = "OCT27"
pi1Mass = 0.13957018
k1Mass = 0.493677
mass1, mass2 = 0, 0
chainSGN_Phi3 = ROOT.TChain("events")
chainSGN_Phi3.Add("/data/submit/pdmonte/outputs/{0}/2018/outname_mc{1}_GFcat_{2}_2018.root".format(date, 1039, "Phi3Cat"))
dfSGN_Phi3 = ROOT.RDataFrame(chainSGN_Phi3)
dfSGN_Phi3 = (dfSGN_Phi3.Define("scale", "w*lumiIntegrated")
                .Define("newDitrackMass", "sum2Body(goodMeson_trk1_pt[0], goodMeson_trk1_eta[0], goodMeson_trk1_phi[0], {}, goodMeson_trk2_pt[0], goodMeson_trk2_eta[0], goodMeson_trk2_phi[0], {}).M()".format(pi1Mass, pi1Mass))
                .Define("DiffFittedMass", "goodMeson_ditrk_mass - goodMeson_ditrk_mass_GEN")
                .Define("DiffSumMass", "newDitrackMass - goodMeson_ditrk_mass_GEN")
                .Define("DiffRawMass", "goodMeson_mass_raw - goodMeson_mass_GEN")
                .Define("DiffModifiedMass", "goodMeson_mass - goodMeson_mass_GEN")
                .Define("Residual_ditrk_pt", "goodMeson_ditrk_pt - goodMeson_ditrk_pt_GEN")
                .Define("Residual_old_pt", "goodMeson_pt - goodMeson_pt_GEN")
                .Define("Residual_ditrk_mass", "goodMeson_ditrk_mass - goodMeson_ditrk_mass_GEN")
                .Define("Residual_ditrk_eta", "goodMeson_ditrk_eta - goodMeson_ditrk_eta_GEN")
                .Define("Residual_ditrk_phi", "goodMeson_ditrk_phi - goodMeson_ditrk_phi_GEN"))

In [ ]:
defaultHistColors = [ROOT.kRed + 1, ROOT.kBlue, ROOT.kGreen + 2, ROOT.kYellow + 1, ROOT.kOrange + 8]

def savePlot(histograms, options=None):
    cs = ROOT.TCanvas("canvas", "canvas", 900, 900)

    stack4 = ROOT.THStack()
    legend4 = ROOT.TLegend(0.70, 0.70, 0.899, 0.899)

    usedColors = options["colors"] if "colors" in options else defaultHistColors

    markers = []
    maxHeight = 0
    for i, h in enumerate(histograms):
        maxHeight = max(maxHeight, h.GetMaximum())
        if "style" in options:
            if options["style"][i] == "l":
                stack4.Add(h[1])
                legend4.AddEntry(h[1], h[0], "l")
                h[1].SetLineWidth(3)
                h[1].SetLineColor(usedColors[i])
            elif options["style"][i] == "f":
                legend4.AddEntry(h[1], h[0], "f")
                stack4.Add(h[1])
                h[1].SetLineWidth(0)
                h[1].SetFillColor(usedColors[i])
            elif options["style"][i] == "p":
                legend4.AddEntry(h[1], h[0], "lep")
                markers.append((h[1], usedColors[i]))
        else:
            stack4.Add(h[1])
            legend4.AddEntry(h[1], h[0], "l")
            h[1].SetLineWidth(3)
            h[1].SetLineColor(usedColors[i])

    stack4.SetMaximum(1.2*maxHeight)

    if "HCandMass" in options:
        stack4.Draw("hist stack")
        line1 = ROOT.TLine(115., 0., 115., 500.)
        line1.SetLineColor(11)
        line1.Draw()
        line2 = ROOT.TLine(135., 0., 135., 500.)
        line2.SetLineColor(11)
        line2.Draw()
    else:
        stack4.Draw("hist nostack")

    for h, col in markers:
        h.SetMarkerStyle(20)
        h.SetMarkerSize(1.3)
        h.SetLineWidth(3)
        h.SetMarkerColor(col)
        h.SetLineColor(col)
        h.Draw("EP SAME")

    if "labelXAxis" in options:
        stack4.GetXaxis().SetTitle(options["labelXAxis"])
    if "labelYAxis" in options:
        stack4.GetYaxis().SetTitle(options["labelYAxis"])
    if "xRange" in options:
        stack4.GetXaxis().SetRangeUser(options["xRange"][0], options["xRange"][1])
    if "logScale" in options:
        if options["logScale"]:
            stack4.SetMaximum(10*maxHeight)
            cs.SetLogy(1)
    
    #legend4.SetMargin(0.17)
    legend4.SetBorderSize(0)
    legend4.SetTextSize(0.035)
    legend4.Draw()

    text = ROOT.TLatex()
    text.SetNDC()
    text.SetTextFont(72)
    text.SetTextSize(0.045)
    text.DrawLatex(0.105, 0.913, "CMS")
    text.SetTextFont(42)
    text.DrawLatex(0.105 + 0.12, 0.913, "Internal" if options["data"] else "Simulation")
    text.SetTextSize(0.035)
    text.DrawLatex(0.59, 0.913, "#sqrt{s} = 13 TeV, %0.2f fb#kern[-0.1]{^{-1}}"% (39.54))

    cs.SaveAs("/home/submit/pdmonte/public_html/thesisPlots/test.png")

In [ ]:
df = dfSGN_Phi3
options = {"labelXAxis": "m_{{#gamma{}}} [GeV]".format("#phi"), "labelYAxis": "Events", "style": ["f", "f", "p"], "colors": [ROOT.kOrange - 9, ROOT.kRed + 2, ROOT.kBlack], "data": False}
nbins, xlow, xhigh = 100, 0, 1.2
histograms = []
name1 = "signal"
h1 = df.Histo1D(("hist", name1, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
name2 = "backgriund"
h2 = df.Histo1D(("hist", name2, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
name3 = "data"
h3 = df.Histo1D(("hist", name3, nbins, xlow, xhigh), "goodMeson_ditrk_mass_GEN", "scale").GetValue()
h1.Scale(1.5)
h2.Scale(1.2)
h3.Scale(1.8)
histograms.append((name1, h1))
histograms.append((name2, h2))
histograms.append((name3, h3))
a = savePlot(histograms, options=options)

Info in <TCanvas::Print>: png file /home/submit/pdmonte/public_html/thesisPlots/test.png has been created
